# **Семинар 1. Технический мониторинг с помощью Prometheus и Grafana**

---

## **Цель семинара**

Познакомиться с инструментами мониторинга инфраструктуры ML-сервисов и научиться собирать, хранить и визуализировать метрики с помощью **Prometheus** и **Grafana**.

После занятия студент сможет:

- добавлять системные и пользовательские метрики в код ML-сервиса;  
- настраивать сбор данных Prometheus;  
- визуализировать метрики в Grafana;  
- задавать пороговые алерты и анализировать состояние модели.

---


In [ ]:
# 1. Подготовка окружения

!pip install fastapi uvicorn prometheus_client
!docker pull prom/prometheus
!docker pull grafana/grafana

/bin/bash: line 1: docker: command not found
/bin/bash: line 1: docker: command not found


## **2. Создание простого ML-сервиса с метриками**

Создадим FastAPI-приложение, которое будет:

- считать количество запросов к эндпоинту `/predict`;
- измерять задержку (latency);
- подсчитывать количество ошибок;
- экспортировать метрики на порт `8001` для Prometheus.


In [ ]:
from fastapi import FastAPI
from prometheus_client import Counter, Histogram, start_http_server
import time, os, random

app = FastAPI(title="ML Monitoring Demo")

# --- метрики Prometheus ---
REQUEST_COUNT = Counter("predict_requests_total", "Total number of prediction requests")
LATENCY = Histogram("prediction_latency_seconds", "Model prediction latency (s)")
ERROR_COUNT = Counter("predict_errors_total", "Total number of prediction errors")

# --- версия модели ---
VERSION = os.getenv("MODEL_VERSION", "v1.0.0")

@app.get("/health")
def health():
    return {"status": "ok", "version": VERSION}

@app.get("/predict")
def predict():
    start = time.time()
    REQUEST_COUNT.inc()
    try:
        # имитируем задержку и случайную ошибку
        if random.random() < 0.1:
            raise ValueError("Random failure!")
        time.sleep(random.uniform(0.05, 0.3))
        LATENCY.observe(time.time() - start)
        return {"prediction": "class_A", "version": VERSION}
    except Exception as e:
        ERROR_COUNT.inc()
        LATENCY.observe(time.time() - start)
        return {"error": str(e), "version": VERSION}

# сервер метрик Prometheus (порт 8001)
start_http_server(8001)


In [ ]:
# запуск приложения локально
uvicorn main:app --host 0.0.0.0 --port 8000


После запуска:

- ML-сервис доступен по адресу: http://localhost:8000  
- метрики — на http://localhost:8001/metrics  


## **3. Настройка Prometheus**

Создаём файл `prometheus.yml` в корне проекта:


global:
  scrape_interval: 5s

scrape_configs:
  - job_name: "ml_service"
    static_configs:
      - targets: ["host.docker.internal:8001"]


In [ ]:
# запуск Prometheus
docker run -d --name prometheus \
  -p 9090:9090 \
  -v $(pwd)/prometheus.yml:/etc/prometheus/prometheus.yml \
  prom/prometheus


Prometheus UI: http://localhost:9090  
Попробуем выполнить запрос:
predict_requests_total


## **4. Настройка Grafana**

Grafana собирает данные из Prometheus и визуализирует их.


In [ ]:
# запуск Grafana
docker run -d -p 3000:3000 --name=grafana grafana/grafana

1. Откройте http://localhost:3000  
   Вход: **admin / admin**

2. Добавьте источник данных:
   - Type: Prometheus  
   - URL: `http://host.docker.internal:9090`

3. Создайте Dashboard с панелями:
   - `rate(predict_requests_total[1m])` — частота запросов;  
   - `histogram_quantile(0.95, sum(rate(prediction_latency_seconds_bucket[1m])) by (le))` — 95-й перцентиль задержки;  
   - `increase(predict_errors_total[5m])` — число ошибок за 5 минут.


## **5. Настройка алертов**

Создайте alert в Grafana:

**Условие:**  
`avg(prediction_latency_seconds_sum / prediction_latency_seconds_count) > 1`

**Действие:**  
Slack / Email-уведомление или визуальный триггер.

---


## **6. Интерактивные вопросы**

**Почему нельзя использовать print-логи вместо Prometheus-метрик?**  
Логи — неструктурированные данные, их трудно агрегировать; метрики позволяют отслеживать состояние сервиса в реальном времени.

**Какие метрики особенно важны для ML-моделей?**  
Latency, error rate, CPU/RAM usage, количество запросов, drift accuracy, confidence score.


## **7. Мини-практикум**

1. Добавьте новую метрику `MODEL_CONFIDENCE` — Histogram распределения уверенности модели.  
2. Выведите среднюю уверенность на дашборде Grafana.  
3. Настройте alert, если средняя уверенность < 0.6.  


## **8. Итог семинара**

После занятия вы получили:

- ML-сервис с эндпоинтами `/predict` и `/metrics`;  
- настроенный Prometheus для сбора метрик;  
- Grafana-дашборд с графиками и алертами.  

Мониторинг — ключевой элемент MLOps:  
он позволяет вовремя обнаруживать деградацию модели, перегрузку сервера и ошибки деплоя.
